In [12]:
import skimage 
from skimage import data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torchvision
import torchvision.transforms as transforms

from torchvision import datasets
#dir(transforms)

['CenterCrop',
 'ColorJitter',
 'Compose',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'functional',
 'transforms']

In [27]:
data_csv = pd.read_csv('./Data_Entry_2017.csv')

#col_one_list = data_csv[data_csv['Finding Labels'].eq('Cardiomegaly')]
#Cardiomegaly_list = col_one_list['Image Index'].values.tolist()
#len(Cardiomegaly_list)

total_disease_list = ['Cardiomegaly', 'No Finding', 'Nodule', 'Emphysema', 'Effusion', 'Infiltration',
                     'Hernia', 'Mass', 'Fibrosis', 'Pleural_Thickening', 'Consolidation',
                     'Pneumothorax', 'Atelectasis', 'Edema', 'Pneumonia']


image_list = {}
sum = 0

for disease in total_disease_list:
    col_one_list = data_csv[data_csv['Finding Labels'].eq(disease)]
    image_list[disease] = col_one_list['Image Index'].values.tolist()
    #sum = sum + len(image_list[disease])
    #print("Disease Name: ", disease)
    #print("Total Value Per Each Disease: ", len(image_list[disease]))
    
#print("Total : ", sum)

def getLabel( image_name, dict_image_list ):
    MULTI = 14
    for i, (key, value) in enumerate(dict_image_list.items()):
        for img_name in value:
            if( img_name == image_name ):
                return i
            
    return MULTI

#print("Datat : ", image_list)
            
#label = getLabel('00001941_007.png', image_list ) 
#label


#data_to_exclude = ['00000001_000.png']


class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    #print("path : ")
    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        #print("inside : ")
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        label = getLabel(path.split('/')[-1], image_list)
        #if( path.split('/')[-1] not in data_to_exclude ):
            #print("path : ", path)
            #print("Original tuple : ", original_tuple)
            # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple+ (label,))
        return tuple_with_path

In [28]:
size_1 = (512,512)
size_2 = (256,256)
batch_size = 5
num_workers = 2

In [29]:
transform = transforms.Compose([
    transforms.Resize(size=size_1),
    transforms.ToTensor()
])

In [31]:
data_dir = 'cs535-Image/images/'
dataset = ImageFolderWithPaths(root= data_dir,transform = transform) # our custom dataset
dataloader_1 = torch.utils.data.DataLoader(dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=num_workers)


dataset



Dataset ImageFolderWithPaths
    Number of datapoints: 15
    Root location: cs535-Image/images/
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [35]:
image_batch, labels_batch, labels = iter(dataloader_1).next()
image_batch.shape, labels_batch, labels

(torch.Size([5, 3, 512, 512]),
 tensor([0, 0, 0, 0, 0]),
 tensor([ 1,  6,  1, 14,  6]))

In [33]:
img = torchvision.utils.make_grid(image_batch)

img.shape

torch.Size([3, 516, 2572])

In [ ]:
img_1 = np.transpose(img, (1,2,0))
img_1.shape

In [ ]:
plt.figure(figsize=(32,24))

plt.imshow(img_1)
plt.axis('off')
plt.show()

In [ ]:

pop_mean = []
pop_std = []



for i, data in enumerate(dataloader_1):
    #Extract Images at field position 0: 
    #print("batch Count: ", i)
    numpy_image = data[0].numpy()
    #print("batch Count:numpy_image", numpy_image.shape)
    # axis=(0,2,3) => (B, H, W)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std = np.std(numpy_image, axis=(0,2,3))
    #batch_mean = np.mean(numpy_image)
    #batch_std = np.std(numpy_image)
    
    pop_mean.append(batch_mean)
    pop_std.append(batch_std)

In [ ]:

pop_mean = np.array(pop_mean)
pop_std = np.array(pop_std)

# (Total Batchs, ( Mean_Value , Std_Value ) => For each channel )
pop_mean.shape, pop_std.shape 

In [ ]:
# DO NOT RUN - Original Version -> Already Done At the top

pop_mean = pop_mean.mean(axis =0 )
pop_std = pop_std.mean(axis =0 )

# To Print MEAN across all the batches and standard Deviation

print("Mean across all the batches: ",pop_mean)
print("standard Deviation across all the batches: ",pop_std)

In [ ]:
pop_mean, pop_std

In [ ]:
transform = transforms.Compose([
    transforms.Resize(size=size_2),
    transforms.Grayscale(3),
    transforms.ToTensor()
])


#transforms.Normalize(mean=pop_mean,
#                         std=pop_std)

In [ ]:
dataset_train = torchvision.datasets.ImageFolder(root='cs535-Image/Covid-Dataset/dataset/', transform = transform)
dataset_train

In [ ]:
# LABELS

dataset_train.class_to_idx, dataset_train.classes

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [60, 20, 18])


In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

In [ ]:
dataloader = torch.utils.data.DataLoader(train_set,
                                        batch_size=5,
                                        shuffle=False,
                                        num_workers=2)


dataloader

In [ ]:
image_batch, label_batch = iter(dataloader).next()
image_batch.shape

In [ ]:
img = torchvision.utils.make_grid(image_batch,nrow=5)

In [ ]:
img = np.transpose(img, (1,2,0))
img.shape

In [ ]:
m, M = img.min(), img.max()

m, M

In [ ]:
#img = (1 / abs(m) * M ) * img + 0.3

In [ ]:


plt.figure(figsize=(16, 12))

plt.imshow(img)
plt.show()